Followed tutorial from https://www.stratascratch.com/blog/step-by-step-guide-to-building-content-based-filtering

This movie recommender only use summary of movies.

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
from copy import deepcopy as dp

In [3]:
#reading the file
movie_data = pd.read_csv('./the_movies_dataset/movies_metadata.csv')

C:\Users\dokha\AppData\Local\Temp\ipykernel_40508\1399212188.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_data = pd.read_csv('./the_movies_dataset/movies_metadata.csv')


In [4]:
movie_data.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [5]:
movie_data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [6]:
movie_data.iloc[0]

adult                                                                False
belongs_to_collection    {'id': 10194, 'name': 'Toy Story Collection', ...
budget                                                            30000000
genres                   [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
homepage                              http://toystory.disney.com/toy-story
id                                                                     862
imdb_id                                                          tt0114709
original_language                                                       en
original_title                                                   Toy Story
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                       21.946943
poster_path                               /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
production_companies        [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries     

I need genres (list[dict[]]) after conversion, popularity: float, runtime: float, spoken_languages: (list[dict[]]) -> after conversion, vote_average

In [7]:
# Movie summary data, fill null value with empty string
movie_summary = movie_data['overview'].fillna('')

# Movie title data
movie_title_data = movie_data['original_title']

# Map movie title to its index
movie_to_index = pd.Series(movie_data.index, index=movie_data['title']).drop_duplicates()

Convert movie_genres into list[dict[]] for easier use

In [8]:
from ast import literal_eval

In [9]:
#Movie genre
movie_genres = movie_data['genres'].fillna('[{}]')

In [10]:
def convert_string_list_dic_into_actual(data):
    for i in range(len(data)):
        temp = data[i].strip('[').strip(']').replace("{", "").replace("}", "").split(',')
    
        convert_version = []
        for _ in range(int(len(temp) // 2)):
            convert_version.append(literal_eval("{" + f"{temp[0]}" + "," +f"{temp[1]}" + "}"))
            temp = temp[2:]
        data[i] = convert_version
    

In [11]:
#Movie language
movie_spoken_language = movie_data['spoken_languages'].fillna('[]')

In [12]:
convert_string_list_dic_into_actual(movie_genres)
convert_string_list_dic_into_actual(movie_spoken_language)

In [13]:
movie_spoken_language

0                 [{'iso_639_1': 'en', 'name': 'English'}]
1        [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
2                 [{'iso_639_1': 'en', 'name': 'English'}]
3                 [{'iso_639_1': 'en', 'name': 'English'}]
4                 [{'iso_639_1': 'en', 'name': 'English'}]
                               ...                        
45461               [{'iso_639_1': 'fa', 'name': 'فارسی'}]
45462                    [{'iso_639_1': 'tl', 'name': ''}]
45463             [{'iso_639_1': 'en', 'name': 'English'}]
45464                                                   []
45465             [{'iso_639_1': 'en', 'name': 'English'}]
Name: spoken_languages, Length: 45466, dtype: object

Let's just form a concatenated string of genres together so that I can use TF-IDF for values.

In [14]:
#Function for concatenating string of genres or languages together
def concate_string_list_dict_into_simple_string(data):
    for index in range(len(data)):
        temp_string = ""
        for element in data.iloc[index]:
            temp_string += f"{element['name']} "
        data.iloc[index] = temp_string

In [15]:
concate_string_list_dict_into_simple_string(movie_genres)
concate_string_list_dict_into_simple_string(movie_spoken_language)

In [16]:
#We will concatenate the three text-based reviews together to form a big variable so that we can use tfidf to produce item_feature matrix
movie_str_criteria = dp(movie_genres)
for index in range(len(movie_genres)):
    movie_str_criteria[index] += movie_genres[index] + movie_spoken_language[index]

In [17]:
tfidf = TfidfVectorizer(stop_words='english')
movie_matrix = tfidf.fit_transform(movie_str_criteria)

In [18]:
movie_matrix.shape

(45466, 120)

In [19]:
numerical_data = movie_data[['runtime', 'popularity', 'vote_average']]

In [20]:
numerical_data = numerical_data.fillna(0.0)

In [34]:
numerical_data['popularity'].map(type).value_counts()

popularity
<class 'float'>    32770
<class 'str'>      12697
Name: count, dtype: int64

In [35]:
numerical_data['popularity'].dtypes

dtype('O')

In [24]:
numerical_data.astype({'popularity':float}, copy = False, errors = 'ignore')

,runtime,popularity,vote_average
0,81.0,21.946943,7.7
1,104.0,17.015539,6.9
2,101.0,11.7129,6.5
3,127.0,3.859495,6.1
4,106.0,8.387519,5.7
...,...,...,...
45461,90.0,0.072051,4.0
45462,360.0,0.178241,9.0
45463,90.0,0.903007,3.8
45464,87.0,0.003503,0.0


In [33]:
numerical_data.loc[numerical_data['popularity'].dtypes,['popularity']] = float(0)

In [18]:
from sklearn.preprocessing import StandardScaler

In [19]:
scaler = StandardScaler()
scaled_numerical_data = scaler.fit_transform(numerical_data)

In [20]:
from scipy.sparse import hstack

In [ ]:
movie_matrix_calculated = hstack[movie_matrix,scaled_numerical_data]

In [ ]:
title = 'Furious 7'
idx = movie_to_index[title]

In [ ]:
this_movie_criteria = [movie_str_criteria[idx]]

In [ ]:
this_movie_matrix = tfidf.transform(this_movie_criteria)

In [ ]:
this_movie_matrix['runtime'] = movie_data[idx]['runtime']
this_movie_matrix['popularity'] = movie_data[idx]['popularity']
this_movie_matrix['vote_average'] = movie_data[idx]['vote_average']

In [ ]:
sim_scores = cosine_similarity(this_movie_matrix, movie_matrix).tolist()[0]

In [ ]:
#stop_words we can ignore are the ones in English
tfidf = TfidfVectorizer(stop_words='english')
#result of training the tfidf model is that we will get the item-feature matrix.
# movie_matrix= tfidf.fit_transform(movie_str_criteria)
movie_matrix = tfidf.fit_transform(movie_summary)

In [8]:
movie_matrix

AttributeError: 'csr_matrix' object has no attribute 'head'

In [ ]:
# Template 
# zz = movie_genres[0].strip('[').strip(']')
# zz = zz.replace("{", "").replace("}", "").split(',')

In [ ]:
#convert movie

In [122]:
movie_summary[0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [5]:
movie_matrix.shape

(45466, 75827)

#each movie is a vector of size 75827.
We are using cosine to find similarity. (we could use Euclidean or dot product)

Movie_summary is the key feature to determine which movie to recommend.
Use TF-IDF to process text form of the movie summary into numerical so you can use cosine to find similarity

In [6]:
title = 'Furious 7'
idx = movie_to_index[title]

In [10]:
this_movie_summary = [movie_summary[idx]]

In [11]:
this_movie_matrix = tfidf.transform(this_movie_summary)

In [12]:
print(this_movie_matrix.shape)

(1, 75827)


In [68]:
sim_scores = cosine_similarity(this_movie_matrix, movie_matrix).tolist()[0]

In [69]:
# There are 45466 movies that we have computed their cosine_similarity
# enumerate object looks like (n, value) where n is number of instances and value is result of cosine 
# similarity
# reverse = True means sort in descing order
# The integrity of the movie index in movie_indexes and movie_matrix are kept since we didn't change the
# order in movie_matrix.
sim_scores = sorted(enumerate(sim_scores), key= lambda i: i[1], reverse = True)

# Fetch the top 10 recommended movies
sim_scores = sim_scores[1:11]

In [70]:
print(sim_scores)

[(4241, 0.2840828188953829), (20970, 0.2271265118239757), (40993, 0.21878271091976215), (17052, 0.1951022599960312), (29744, 0.17015267141759066), (12506, 0.17012985750201387), (44235, 0.15341519203643178), (34139, 0.15195480336228978), (3739, 0.1516962746883741), (13442, 0.1484508861061471)]


In [71]:
movie_indexes = [i[0] for i in sim_scores]

In [74]:
print([movie_title_data[i] for i in movie_indexes])

['The Fast and the Furious', 'Fast & Furious 6', 'Los violadores', 'Fast Five', 'Genius on Hold', 'Youth Without Youth', 'The Skydivers', 'Aenigma', 'The Cell', 'Urban Justice']


In [81]:
def get_recommendation(movie_title: str):
    index = movie_to_index[movie_title]
    this_movie_summary = [movie_summary[index]]
    this_movie_matrix = tfidf.transform(this_movie_summary)
    sim_scores = cosine_similarity(this_movie_matrix, movie_matrix).tolist()[0]
    sim_scores = sorted(enumerate(sim_scores), key = lambda t: t[1], reverse = True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    movie_titles = [movie_title_data[i] for i in movie_indices ]

    return movie_titles
    

In [82]:
get_recommendation("Casablanca")

['Delta of Venus',
 'Desirable',
 'The Sun Also Rises',
 'Chloe',
 'White Nights',
 'The Russia House',
 'Anthropoid',
 'The Wind and the Lion',
 'Mille Mois',
 'A Night in Casablanca']